# Testing Pipelines

### Initiating observation via Phoenix

In [1]:
# %pip pip install arize-phoenix
# %pip install llama-index-callbacks-arize-phoenix
# observability
import llama_index.core  # type: ignore
import phoenix as px  # type: ignore

px.launch_app()

llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
example_inputs = [
    "Current climate models (CMIP6) have difficulties in representing clouds.",
    "Due to climate change, there is less spring snow cover in Europe.",
    "Northern Hemisphere spring snow cover is decreasing because of anthropogenic climate change.",
    "Carbon dioxide removal (CDR) has no substantial side-effects.", # 99
    "Carbon dioxide removal (CDR) is highly effective and safe.",
    "Climate change is the greatest threat to humanity.",
    "An immediate global ban of fossil fuels is the absolutely only way to avoid climate armageddon.",
]

In [4]:
from evidence_seeker import EvidenceSeeker
from evidence_seeker.retrieval import RetrievalConfig

retrieval_config = RetrievalConfig()
retrieval_config.document_input_files = ["../TMP/IPCC_AR6_WGI_TS.pdf"]
retrieval_config.index_persist_path = "../TMP/storage/index"

pipeline = EvidenceSeeker(retrieval_config=retrieval_config)

2024-12-13 11:33:35.959 | DEBUG    | evidence_seeker.backend:get_openai_llm:109 - Fetching api key via env var: HF_TOKEN_EVIDENCE_SEEKER
2024-12-13 11:33:35.962 | DEBUG    | evidence_seeker.backend:get_openai_llm:117 - Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://ev6086dt6s7nn1b5.us-east-1.aws.endpoints.huggingface.cloud/v1/).
2024-12-13 11:33:41.265 | DEBUG    | evidence_seeker.backend:get_openai_llm:109 - Fetching api key via env var: HF_TOKEN_EVIDENCE_SEEKER
2024-12-13 11:33:41.266 | DEBUG    | evidence_seeker.backend:get_openai_llm:117 - Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://ev6086dt6s7nn1b5.us-east-1.aws.endpoints.huggingface.cloud/v1/).


In [5]:
from IPython.display import display, Markdown  # type: ignore

def describe_results(claim: str, results: list):
    preamble_template = (
        '## EvidenceSeeker Results\n\n'
        '### Input\n\n'
        '**Submitted claim:** {claim}\n\n'
        '### Results\n\n'
    )
    result_template = (
        '**Clarified claim:** <font color="orange">{text}</font> [_{statement_type}_]\n\n'
        '**Status**: {verbalized_confirmation}\n\n'
        '|Metric|Value|\n'
        '|:---|---:|\n'
        '|Average confirmation|{average_confirmation:.3f}|\n'
        '|Evidential divergence|{evidential_uncertainty:.3f}|\n'
        '|Width of evidential base|{n_evidence}|\n\n'
    )
    markdown = []
    markdown.append(preamble_template.format(claim=claim))
    for claim_dict in results:
        rdict = claim_dict.copy()
        rdict["statement_type"] = rdict["statement_type"].value
        markdown.append(result_template.format(**claim_dict))
    display(Markdown("\n".join(markdown)))


In [6]:
import asyncio

coros = [
    pipeline(example_input)
    for example_input in example_inputs
]
results = await asyncio.gather(*coros)
for input, output in zip(example_inputs, results):
    describe_results(input, output)
    display(Markdown("------\n"))

2024-12-13 11:33:50.707 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.


2024-12-13 11:33:50.754 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024-12-13 11:33:50.759 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:217 - Analysing normative aspects of claim.
2024-12-13 11:33:50.762 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.
2024-12-13 11:33:50.767 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024-12-13 11:33:50.772 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:217 - Analysing normative aspects of claim.
2024-12-13 11:33:50.776 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.
2024-12-13 11:33:50.781 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024

## EvidenceSeeker Results

### Input

**Submitted claim:** Current climate models (CMIP6) have difficulties in representing clouds.

### Results


**Clarified claim:** <font color="orange">Current climate models should perform better in representing clouds.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">CMIP6 climate models exist and are a current generation of climate models.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|7|


**Clarified claim:** <font color="orange">CMIP6 climate models have limitations or biases in simulating clouds.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|3|


**Clarified claim:** <font color="orange">Cloud representation is a challenging aspect of climate modeling in CMIP6.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|3|



------


## EvidenceSeeker Results

### Input

**Submitted claim:** Due to climate change, there is less spring snow cover in Europe.

### Results


**Clarified claim:** <font color="orange">Climate change is occurring.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|8|


**Clarified claim:** <font color="orange">There is a decrease in spring snow cover in Europe.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">The decrease in spring snow cover is caused by climate change.</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|5|



...

------


## EvidenceSeeker Results

### Input

**Submitted claim:** Climate change is the greatest threat to humanity.

### Results


**Clarified claim:** <font color="orange">Human life, health, and prosperity are valuable and worth protecting.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Climate change is a more significant concern than other potential threats to humanity.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">It is desirable to mitigate the negative impacts of climate change to protect human well-being.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|



------


## EvidenceSeeker Results

### Input

**Submitted claim:** An immediate global ban of fossil fuels is the absolutely only way to avoid climate armageddon.

### Results


**Clarified claim:** <font color="orange">Avoiding climate armageddon is a desirable goal.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">An immediate global ban of fossil fuels is a necessary action to achieve a desirable outcome (avoiding climate armageddon).</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">An immediate global ban of fossil fuels is a justifiable or recommended course of action to avoid a catastrophic outcome.</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|



------


### 12/12/2024

In [4]:
for example_input in example_inputs:
    results = await pipeline(example_input)
    describe_results(example_input, results)
    display(Markdown("------\n"))

2024-12-12 17:28:54.926 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.


2024-12-12 17:28:55.020 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024-12-12 17:28:55.024 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:217 - Analysing normative aspects of claim.
2024-12-12 17:29:02.876 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:29:02.882 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:29:03.661 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:29:03.667 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:29:03.672 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
/Users/ggbetz/git/evidence-seeker/.hatch/evidence-seeker/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.


## EvidenceSeeker Results

### Input

**Submitted claim:** Current climate models (CMIP6) have difficulties in representing clouds.

### Results


**Clarified claim:** <font color="orange">Climate models should accurately represent clouds</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Current climate models (CMIP6) are not meeting the expected level of performance in representing clouds</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Representing clouds is a feature of climate models</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|0.954|
|Evidential divergence|0.006|
|Width of evidential base|4|


**Clarified claim:** <font color="orange">Current climate models (CMIP6) face difficulties in accurately representing clouds</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|2|


**Clarified claim:** <font color="orange">Current climate models (CMIP6) exist</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|8|



------


2024-12-12 17:29:57.126 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.
2024-12-12 17:29:57.132 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024-12-12 17:29:57.138 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:217 - Analysing normative aspects of claim.
2024-12-12 17:30:05.340 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:30:05.346 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:30:05.735 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:30:05.741 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:30:06.971 | DEBUG    | evidence_seeker.confirmation_analysis.workflows:freetext_analysis:72 - Confirmation analy

...

## EvidenceSeeker Results

### Input

**Submitted claim:** Climate change is the greatest threat to humanity.

### Results


**Clarified claim:** <font color="orange">The severity of climate change is greater than that of other potential threats to humanity</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Human well-being and safety are of paramount importance and should be prioritized</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Climate change has a significant negative impact on human well-being, safety, or existence</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Climate change exists</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|5|


**Clarified claim:** <font color="orange">Climate change poses a threat to humanity</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|3|


**Clarified claim:** <font color="orange">Climate change has significant negative consequences for human societies, ecosystems, or the planet as a whole</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|1|



------


2024-12-12 17:34:19.085 | DEBUG    | evidence_seeker.preprocessing.workflows:ascriptive_analysis:168 - Analysing ascriptive aspects of claim.
2024-12-12 17:34:19.090 | DEBUG    | evidence_seeker.preprocessing.workflows:descriptive_analysis:119 - Analysing descriptive aspects of claim.
2024-12-12 17:34:19.094 | DEBUG    | evidence_seeker.preprocessing.workflows:normative_analysis:217 - Analysing normative aspects of claim.
2024-12-12 17:34:29.576 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:34:29.580 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:34:29.585 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:34:29.590 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:34:29.801 | DEBUG    | evidence_seeker.preprocessing.workflows:negate_claim:266 - Negating claim.
2024-12-12 17:

## EvidenceSeeker Results

### Input

**Submitted claim:** An immediate global ban of fossil fuels is the absolutely only way to avoid climate armageddon.

### Results


**Clarified claim:** <font color="orange">Fossil fuels contribute to climate change</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|1.000|
|Evidential divergence|0.000|
|Width of evidential base|7|


**Clarified claim:** <font color="orange">No alternative solutions exist to avoid climate armageddon</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">A global ban on fossil fuels is feasible</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">Climate armageddon is a potential consequence of continued fossil fuel use</font> [_StatementType.DESCRIPTIVE_]

**Status**: The claim is strongly confirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|0.802|
|Evidential divergence|0.023|
|Width of evidential base|2|


**Clarified claim:** <font color="orange">Climate armageddon is undesirable and should be avoided</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">The situation requires immediate action to prevent climate armageddon</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|


**Clarified claim:** <font color="orange">A global ban on fossil fuels is a necessary and justified measure to prevent climate armageddon</font> [_StatementType.NORMATIVE_]

**Status**: The claim is neither confirmed nor disconfirmed.

|Metric|Value|
|:---|---:|
|Average confirmation|nan|
|Evidential divergence|nan|
|Width of evidential base|0|



------
